# NeOS Server Log

In [1]:
#date에 뽑고싶은 날짜를 입력
log_date = '2020-12-24'
log_address = ('http://211.110.170.61/log/Ons_SR_'+log_date+'.log')

# 1. 로그 읽어오기 
 - 샘플url : http://211.110.170.61/log/Ons_SR_2020-10-09.log
 - 주의 사항 : 로그 2번 읽으면 json 문구 오류 발생함 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame as df
from pandas import json_normalize
from matplotlib import font_manager, rc
import os

In [3]:
#해당일자의 서버로그를 가져오기, 한글 인코딩
import urllib.request
url = log_address
savename = (log_date+'.log')
mem=urllib.request.urlopen(url).read().decode('cp949')

#현재 시간 타임코드 찍어두기
from datetime import datetime
now = datetime.now()
now=now.strftime('%Y-%m-%d-%H-%M')

In [4]:
#데이터 균질화 : ★★★★★★★★★이거 2번 실행하면 좆댐
mem = mem.replace('true','"true"')
mem = mem.replace('false','"false"')

In [5]:
#라인으로 나누고 Input, Output 파싱
line = mem.split('\r\nE ')
line_input=[]
line_output=[]
inputword = ('"version":"2.0"')
outputword = ('"output":{"prompt"')
for i in line:
    if inputword in i:
        line_input.append(i)
    elif outputword in i:
        line_output.append(i)

In [6]:
#요청통계 분석 목적이므로 여기서부터는 Input만 다룸. JSON형태로 변경
json_input = []
for i in line_input:
    #print(i)
    string = i[0:48]
    replace_string = '{"date_time":{"timestamp":"'+string[0:25]+'"}'
    json_input.append(replace_string+i[48:])
     

# 데이터 테이블로 추출

In [7]:
from tqdm.notebook import tqdm

In [8]:
#제이슨 노멀라이즈로 파싱해 데이터 프레임에 저장
df1 = df()
for i in tqdm(range(len(json_input))):
    df1=df1.append(json_normalize(eval(json_input[i])), ignore_index=True)

# 에러발생시
- 아래 주석을 풀고 활용하기

In [9]:
# #제이슨 노멀라이즈로 파싱해 데이터 프레임에 저장
# df1 = df()
# for i in tqdm(range(0,2526)):
#     df1=df1.append(json_normalize(eval(json_input[i])), ignore_index=True)



# #제이슨 노멀라이즈로 파싱해 데이터 프레임에 저장
# df2 = df()
# for i in tqdm(range(2527, 6457)):
#     df2=df2.append(json_normalize(eval(json_input[i])), ignore_index=True)

# df3 = pd.concat([df1,df2])
# df1 = df3

# 2. 데이터 프레임 만들기

In [10]:
#불필요한 컬럼 제거하고 df2로 저장 
#이건 매번 사용 형태에 따라 데이터가 달라질 수도 있어서 에러날 수 있음. 반드시 제거되어야할 컬럼을 첫번째로 삭제되도록 지속 조정 필요

# df2 = df1.drop(['event.version','context.device.type','context.session.isNew','action.parameters.arrival_time.value','action.parameters.prompt.value','context.session.isPlayBuilderRequest','context.supportedInterfaces.Text.version','context.supportedInterfaces.Extension.version'], axis=1)
# df2 = df2.drop(['action.parameters.result.value','action.parameters.setstation_name.value'], axis=1)
# df2 = df2.drop(['event.playServiceId','context.supportedInterfaces.Extension.data.SetData.RqstPrntDeptCode','context.supportedInterfaces.Extension.data.SetData.RqstDeptCode'], axis=1)
# df2 = df2.drop(['context.supportedInterfaces.Extension.data.BizData.BidQt','event.sessionId','context.supportedInterfaces.Extension.data.SetData.EquipCode','context.supportedInterfaces.Extension.data.SetData.EquipType','context.supportedInterfaces.Extension.data.SetData.Latitd','context.supportedInterfaces.Extension.data.SetData.Longitd','context.supportedInterfaces.Extension.data.SetData.LocalName','context.supportedInterfaces.Extension.data.SetData.TakeTime','context.supportedInterfaces.Extension.data.SetData.ServerIP','context.supportedInterfaces.Extension.data.SetData.MyLatitd','context.supportedInterfaces.Extension.data.SetData.MyLongitd','context.supportedInterfaces.Extension.data.SetData.RegtNo','context.supportedInterfaces.Extension.data.SetData.StartDate','context.supportedInterfaces.Extension.data.BizData.WorkList.OOS'], axis=1)
# df2 = df2.drop(['context.supportedInterfaces.Extension.data.BizData.WorkList.MyWork','context.supportedInterfaces.Extension.data.BizData.WorkList.Scheduler','context.supportedInterfaces.Extension.data.BizData.WorkList.Notice','context.supportedInterfaces.Extension.data.BizData.HistoryList.Alarm','context.supportedInterfaces.Extension.data.BizData.HistoryList.Work','context.supportedInterfaces.Extension.data.BizData.HistoryList.Access','context.supportedInterfaces.Extension.data.BizData.HistoryList.WorkLog','context.supportedInterfaces.Extension.data.SetData.WorkDvsCode','context.supportedInterfaces.Extension.data.BizData.EquipList'], axis=1)


#주요 요청 정보인 Type과 Value를 신규생성
#Nan 값을 다 비움
df2=df1.fillna('')

#입력 타입을 합치기 위해 type으로 끝나는 것만 df3으로 만들어서 RequestType으로 추가
#df2=df1
df3=df2.filter(regex='type$', axis=1)
df2['RequestType'] = df3.apply('/'.join, axis=1)

#RequestValue df4로 만들어서 df2에 추가
df4=df2.filter(regex='value$', axis=1)
df2['RequestValue'] = df4.apply('/'.join, axis=1)

#불필요한 / 제거
df2['RequestType'] = df2['RequestType'].str.replace(pat=r'[/]', repl= r' ', regex=True)  # replace all special symbols to space
df2['RequestType'] = df2['RequestType'].str.replace(pat=r'\s+', repl= r' ', regex=True)  # replace multiple spaces with a single space
df2['RequestValue'] = df2['RequestValue'].str.replace(pat=r'[/]', repl= r' ', regex=True)  # replace all special symbols to space
df2['RequestValue'] = df2['RequestValue'].str.replace(pat=r'\s+', repl= r' ', regex=True)  # replace multiple spaces with a single space
df2['RequestType']=df2['RequestType'].str.strip()
df2['RequestValue']=df2['RequestValue'].str.strip()

df2[['RequestType','RequestValue']]



#action.actionName과 RequestType을 이용해 요청 기능 분석해 Function에 저장
conditionlist = [
    (df2['action.actionName'] == "01open_door"),
    (df2['action.actionName'] == '02register_work'),
    (df2['action.actionName'] == '03ask_visitor'),
    (df2['action.actionName'] == '04ask_accessinfo'),    
    (df2['action.actionName'] == '05menu_workhistory'),
    (df2['action.actionName'] == '06navigation'),
    (df2['action.actionName'] == '07change_station'),
    (df2['action.actionName'] == '08go'),
    (df2['action.actionName'] == '09common_nogo_popupclose'),
    (df2['action.actionName'] == '11common_searchresult_number'),
    (df2['action.actionName'] == '13ask_guide'),
    (df2['action.actionName'] == '12ask_who') & ((df2['RequestType']).str.contains ('WORKER')),
    (df2['action.actionName'] == '12ask_who') & ((df2['RequestType']).str.contains ('VACATIONER')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('WORKLIST')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('MYWORK')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('NOTICE')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('SCHEDULER')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('TROUBLE')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('WORKHISTORY')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('ACCESSHISTORY')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('ALARMHISTORY')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('HISTORYINFO')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('WORKLOG')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('VISITOR')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('ACCESSINFO')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('NEARSTATION')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('NEARWORKLIST')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('NEARRU')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('BID_LOC_COUNTY')& ((df2['RequestType']).str.contains ('REPEATER'))),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('BID_LOC_COUNTY')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('BID_LOC_CITY')& ((df2['RequestType']).str.contains ('REPEATER'))),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('BID_LOC_CITY')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('WORKTIME')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('WORKER')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('VACATIONER')),
    (df2['action.actionName'] == '10askinfo') & ((df2['RequestType']).str.contains ('GUIDE')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('WORKLIST')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('MYWORK')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('NOTICE')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('SCHEDULER')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('TROUBLE')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('WORKHISTORY')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('ACCESSHISTORY')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('ALARMHISTORY')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('HISTORYINFO')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('WORKLOG')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('VISITOR')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('ACCESSINFO')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('NEARSTATION')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('NEARWORKLIST')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('NEARRU')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('BID_LOC_COUNTY')& ((df2['RequestType']).str.contains ('REPEATER'))),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('BID_LOC_COUNTY')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('BID_LOC_CITY')& ((df2['RequestType']).str.contains ('REPEATER'))),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('BID_LOC_CITY')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('WORKTIME')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('WORKER')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('VACATIONER')),
    (df2['action.actionName'] == '10common_askinfo') & ((df2['RequestType']).str.contains ('GUIDE'))]
choicelist = ['출입문제어', '작업등록', '출입이력 조회', '출입정보 조회', '업무일지 작성', '국소 길안내', '출동국소 변경', '출동/선택', '확인/닫기','국소검색결과 선택', '도움말', 
              '근무자 조회', '휴가자 조회','오늘의업무 조회', '계획업무 조회', '공지사항 조회', '스케쥴러 조회', 'OOS 조회', '작업이력 조회', '출입이력 조회', '알람이력 조회', 
              '이력정보 조회', '작업일지 조회', '출입이력 조회', '출입정보 조회', '주변기지국 조회', '주변업무 조회', '주변중계기 조회', '중계기검색', '기지국검색','중계기검색(미지원)', '기지국검색(미지원)', '근무시간 조회', '근무자 조회', '휴가자 조회', '도움말',
              '오늘의업무 조회', '계획업무 조회', '공지사항 조회', '스케쥴러 조회', 'OOS 조회', '작업이력 조회', '출입이력 조회', '알람이력 조회', 
              '이력정보 조회', '작업일지 조회', '출입이력 조회', '출입정보 조회', '주변기지국 조회', '주변업무 조회', '주변중계기 조회', '중계기검색', '기지국검색','중계기검색(미지원)', '기지국검색(미지원)', '근무시간 조회', '근무자 조회', '휴가자 조회', '도움말']
df2['Function'] = np.select(conditionlist, choicelist, default='Unknown')


# 엑셀 저장하기

In [11]:
#엑셀로 저장해서 나중에 로데이터 쓸수도 있고, 확인도 하고
#df2.to_excel(log_date+".xlsx")  

#데이터 분석용은 필요한 것만 가져와 processed 파일로 별도 저장

df3=df2[['date_time.timestamp','action.actionName','RequestType','RequestValue','Function','event.text','context.session.id','context.supportedInterfaces.Extension.data.DeviceId','context.supportedInterfaces.Extension.data.SceneId','context.supportedInterfaces.Extension.data.UserId']]

In [12]:
peoplelist = pd.read_excel('./install_yn/통계 대상구분.xlsx', sheet_name='구성원', encoding = 'euc-kr')
event_list = pd.merge(df3,peoplelist, left_on='context.supportedInterfaces.Extension.data.UserId', right_on='사번', how='left')

In [13]:
event_list['action.actionName'].unique()

array(['10askinfo', '14small_event', '09common_nogo_popupclose',
       '05menu_workhistory', '11common_searchresult_number',
       '02register_work', '12ask_who', '08go', '06navigation',
       '01open_door', '10common_askinfo',
       '15register_work_confirmneeded_confirm'], dtype=object)

In [14]:
event_list_v1 = event_list[event_list['action.actionName'] == '14small_event']

In [15]:
# df_deduplication = df2.drop_duplicates(['주차','본부','사번'], keep='first')

event_list_fin = event_list_v1.drop_duplicates(['사번'], keep ='first')

In [16]:
event_list_fin['대상구성원'] = event_list_fin['본부']+'-'+event_list_fin['팀']+'-'+event_list_fin['이름']

<ipython-input-16-040c6e44297e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_list_fin['대상구성원'] = event_list_fin['본부']+'-'+event_list_fin['팀']+'-'+event_list_fin['이름']


In [17]:
event_list_fin.to_excel("./log_data/자동응모/"+log_date+"_processed_자동응모.xlsx")  